In [42]:
import pandas as pd
from uuid import uuid4
import ast
import numpy as np
import itertools

In [43]:
data = pd.read_csv('benj_cowen_290_w_cohere_embeddings2.csv')

In [44]:
data['embeddings'] = data['embeddings'].apply(lambda x: np.array(ast.literal_eval(x)))

In [45]:
data["id"] = [str(uuid4()) for _ in range(len(data))]

In [46]:
data.head()

,Author,title,url,Transcript,Type,Publish_Date,ChunkIndex,tokens,embeddings,id
0,Benjamin Cowen,S&amp;P 500 Outlook,https://www.youtube.com/watch?v=urgVQ-ltTf8,hey everyone and thanks for jumping back into ...,Youtube video,2024-04-23 00:00:00,0,334,"[0.084472656, 0.014717102, 0.005355835, -0.011...",816d87cb-5d12-4066-9ee0-b0c258e698a1
1,Benjamin Cowen,S&amp;P 500 Outlook,https://www.youtube.com/watch?v=urgVQ-ltTf8,market does tend to cool off in about Q3 of th...,Youtube video,2024-04-23 00:00:00,1,325,"[0.03765869, 0.023986816, -0.0019874573, 0.006...",8b033ba1-1e8d-4172-9049-8725960824f3
2,Benjamin Cowen,S&amp;P 500 Outlook,https://www.youtube.com/watch?v=urgVQ-ltTf8,local low the S&P is already up about 2.2% so ...,Youtube video,2024-04-23 00:00:00,2,339,"[0.072509766, 0.043029785, -0.020309448, -0.04...",799152e5-b950-4725-8db8-5d761064f1f8
3,Benjamin Cowen,S&amp;P 500 Outlook,https://www.youtube.com/watch?v=urgVQ-ltTf8,moves up gets a pullback and then get sort of ...,Youtube video,2024-04-23 00:00:00,3,330,"[0.078063965, 0.024459839, -0.0039367676, -0.0...",ad348bbc-8e93-4996-a989-8d94cee075df
4,Benjamin Cowen,S&amp;P 500 Outlook,https://www.youtube.com/watch?v=urgVQ-ltTf8,about this is you can overlay uh you can overl...,Youtube video,2024-04-23 00:00:00,4,360,"[0.09057617, 0.009765625, 0.005191803, -0.0337...",3c51efaf-03f1-4a9f-9e13-6c89a302095c


In [47]:
data.shape

(4408, 10)

In [48]:
len(data.embeddings[0])

1024

In [49]:
import pandas as pd

# Assuming 'data' is your DataFrame
data_dict_list = []

for index, row in data.iterrows():
    data_dict = {
        "id": row['id'],
        "values": row['embeddings'],
        "metadata": {
            "text": row['Transcript'],
            "Author": row['Author'],
            "title": row['title'],
            "url": row['url'],
            "Type": row['Type'],
            "Publish_Date": str(row['Publish_Date']),
        }
    }
    data_dict_list.append(data_dict)



In [50]:
data_dict_list[:3]

[{'id': '816d87cb-5d12-4066-9ee0-b0c258e698a1',
  'values': array([ 0.08447266,  0.0147171 ,  0.00535583, ..., -0.10150146,
         -0.04998779, -0.00912476]),
  'metadata': {'text': "hey everyone and thanks for jumping back into the equity verse today we're going to talk about the S&P 500 and we're just going to be providing a General Market Outlook if you guys like the content make sure you subscribe to the channel give the video a thumbs up and also check out into the cryptoverse premium at into the cryptoverse decom I wanted to just provide some thoughts about the S&P 500 it has been a little while and of course we are coming into that time of the year of pre-election years where the market does tend to cool off just a little bit uh we talked about this in the last video and now you can see that it is starting to transpire a little bit if you're unfamiliar with what I'm talking about if you look at say the year-to-date Roi of the S&P 500 in election years right if you look at the 

In [51]:
from pinecone import ServerlessSpec, PodSpec, Pinecone
spec = ServerlessSpec(cloud='aws', region='us-west-2')

api_key = "805b93a4-7e60-47df-9184-537be20493ce"
# configuring client
pc = Pinecone(api_key=api_key)


import time


In [52]:
pc.delete_index("benjamin-cowen-cohere-raw1")

In [53]:
pc.list_indexes()

{'indexes': []}

In [54]:
# pc.delete_index("benjamin-cowen-cohere-youtube")
# pc.delete_index("benjamin-cowen-youtube")
# pc.delete_index("sample-movies")
# pc.delete_index("youtube-chatbot-agent")

In [55]:
data.shape, len(data_dict_list)

((4408, 10), 4408)

In [56]:
# data_dict_list.to_json('benj_cowen_raw290.jsonl', orient="records", lines=True)
import json

# with open("benj_cowen_raw290.jsonl", "w") as f:
#     for item in data_dict_list:
#         json.dump(item, f)
#         f.write("\n")


with open("benj_cowen_raw290_w_text.jsonl", "w") as f:
    for item in data_dict_list:
        # Convert NumPy arrays to lists
        for key, value in item.items():
            if isinstance(value, np.ndarray):
                item[key] = value.tolist()

        # Serialize the dictionary to JSON and write to file
        json.dump(item, f)
        f.write("\n")


In [57]:
import json

def process_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

jsonl_data = process_jsonl('benj_cowen_raw290_w_text.jsonl')

In [58]:
len(jsonl_data)

4408

In [59]:
import time

index_name = 'benjamin-cowen-cohere-raw1'

dimensions = 1024
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name= index_name,
        dimension = dimensions,
        metric = "cosine",
        spec=spec
        )

# wait for index to be ready before connecting - cell not finish
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [60]:
vector_dim = 1024
vector_count = 4408

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))


with pc.Index('benjamin-cowen-cohere-raw1', pool_threads=30) as index:
    # Send requests in parallel
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(jsonl_data, batch_size=100)
    ]
    [async_result.get() for async_result in async_results]